# BentoML Example:  Deploy to AWS ECS using AWS Fargate

[BentoML](http://bentoml.ai) is an open source framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

This notebook demonstrates how to use BentoML to deploy a machine learning model as a serverless REST API endpoint to AWS ECS. For this demo, we are using the [Sentiment Analysis with Scikit-learn](https://github.com/bentoml/BentoML/blob/master/examples/sklearn-sentiment-clf/sklearn-sentiment-clf.ipynb) example, using dataset from [Sentiment140](http://help.sentiment140.com/for-students/).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=nb&ea=open&el=official-example&dt=deploy-with-aws-ecs)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install bentoml
!pip install sklearn pandas numpy

In [2]:
import bentoml
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

# Prepare Dataset

In [3]:
%%bash

if [ ! -f ./trainingandtestdata.zip ]; then
    wget -q http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    unzip -n trainingandtestdata.zip
fi

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [4]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns

# Model Training

In [5]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df = 100,
                                                        ngram_range = (1,1),
                                                        stop_words = 'english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(dftrain.text, dftrain.polarity)

/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('count_vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=100,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [6]:
Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]
print(classification_report(ytest,sentiment_lr.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83       177
           4       0.82      0.86      0.84       182

   micro avg       0.83      0.83      0.83       359
   macro avg       0.83      0.83      0.83       359
weighted avg       0.83      0.83      0.83       359



In [7]:
sentiment_lr.predict([Xtest[0]])

array([4])

## Create BentoService for model serving

To package this trained model for model serving in production, you will need to create a new BentoML Service by subclassing it:

In [8]:
%%writefile sentiment_lr_model.py

import pandas as pd
import bentoml
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([PickleArtifact('model')])
@bentoml.env(pip_dependencies=['sklearn', 'numpy', 'pandas'])
class SentimentLRModel(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler, typ='series')
    def predict(self, series):
        """
        predict expects pandas.Series as input
        """        
        return self.artifacts.model.predict(series)

Writing sentiment_lr_model.py


## Save BentoService to file archive

In [9]:
# 1) import the custom BentoService defined above
from sentiment_lr_model import SentimentLRModel

# 2) `pack` it with required artifacts
bento_service = SentimentLRModel()
bento_service.pack('model', sentiment_lr)

# 3) save BentoSerivce to file archive
saved_path = bento_service.save()

[2019-12-16 13:29:17,399] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.3. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2019-12-16 13:29:17,401] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.3. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.5.3+19.g4c71912
creating BentoML-0.5.3+19.g4c71912/BentoML.egg-info
creating BentoML-0.5.3+19.g4c71912/bentoml
creating BentoML-0.5.3+19.g4c71912/bentoml/artifact
creating BentoML-0.5.3+19.g4c71912/bentoml/bundler
creating BentoML-0.5.3+19.g4c71912/bentoml/cli
creating BentoML-0.5.3+19.g4c71912/bentoml/clipper
creating BentoML-0.5.3+19.g4c71912/bentoml/configuration
creating BentoML-0.5.3+19.g4c71912/bentoml/deployment
creating BentoML-0.5.3+19.g4c71912/bentoml/deployment/aws_lambda
creating BentoML-0.5.3+19.g4c71912/bentoml/deployment/sagemaker
creating BentoML-0.5.3+19.g4c71912/bentoml/deployment/serverless
creating BentoML-0.5.3+19.g4c71912/bentoml/handlers
creating BentoML-0.5.3+19.g4c71912/bentoml/migrations
creating BentoML-0.5.3+19.g4c71912/bentoml/migrations/versions
creating BentoML-0.5.3+19.g4c71912/bentoml/proto
creating BentoML-0.5.3+19.g4c71912/bentoml/repository
creating BentoML-0.5.3+19.g4c71912/bentoml/server
creating BentoML-0.5.3+19.g4c71912/bentoml

copying bentoml/server/__init__.py -> BentoML-0.5.3+19.g4c71912/bentoml/server
copying bentoml/server/bento_api_server.py -> BentoML-0.5.3+19.g4c71912/bentoml/server
copying bentoml/server/bento_sagemaker_server.py -> BentoML-0.5.3+19.g4c71912/bentoml/server
copying bentoml/server/gunicorn_config.py -> BentoML-0.5.3+19.g4c71912/bentoml/server
copying bentoml/server/gunicorn_server.py -> BentoML-0.5.3+19.g4c71912/bentoml/server
copying bentoml/server/utils.py -> BentoML-0.5.3+19.g4c71912/bentoml/server
copying bentoml/server/static/swagger-ui-bundle.js -> BentoML-0.5.3+19.g4c71912/bentoml/server/static
copying bentoml/server/static/swagger-ui.css -> BentoML-0.5.3+19.g4c71912/bentoml/server/static
copying bentoml/utils/__init__.py -> BentoML-0.5.3+19.g4c71912/bentoml/utils
copying bentoml/utils/cloudpickle.py -> BentoML-0.5.3+19.g4c71912/bentoml/utils
copying bentoml/utils/hybirdmethod.py -> BentoML-0.5.3+19.g4c71912/bentoml/utils
copying bentoml/utils/log.py -> BentoML-0.5.3+19.g4c71912

## Load saved BentoService

In [10]:
# Load exported bentoML model archive from path
loaded_bento_service = bentoml.load(saved_path)

# Call predict on the restored BentoService
loaded_bento_service.predict(pd.Series(["hello", "hi"]))

[2019-12-16 13:30:00,510] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.3. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2019-12-16 13:30:00,523] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.3,  but loading from BentoML version 0.5.3+19.g4c71912
[2019-12-16 13:30:00,526] WARNING - Module `sentiment_lr_model` already loaded, using existing imported module.
[2019-12-16 13:30:00,777] WARNING - BentoML local changes detected - Local BentoML repository including all code ch

array([4, 4])

In [11]:
bento_tag = '{name}:{version}'.format(name=bento_service.name, version=bento_service.version)
print(bento_tag)

SentimentLRModel:20191216132917_85D657


# AWS ECS Deployment

## Build and push docker image to AWS ECR

*Get docker login info from AWS ecr*

In [18]:
!aws ecr get-login --region us-west-2 --no-include-email

docker login -u AWS -p eyJwYXlsb2FkIjoiYnFQdjVIUkRpbzl0bXB4aFM4Ny9nbWp4OTV2UTZwTFp1WEhrNEZyWGsvaHBFNjBqUGU2Vm40aVNPdVhnb3BNaHNZMHJWeFFnQ09vUVBMRHdIbXdTSDR2TVZCdTUrL0gwb3V0Z3dJRFMwUUx6MmNxSmdPK0pqUmh4SWhDZTIwa0dHWWZ3M3gvM0pxYXlTcUdnUUtkSGFhMDJkenBnTWNOVU9ia2NESHJKQ0NJeGNZUGgwWHhiNWpCT0piYTlrU0RwajJIVFJCalNnMjZaRnZJUWVwcnprY0JpNStiUnMzVWFmbXozdHlHczJzTzI1SXh1QjFudzBiOVNIdjBXZTZydWVDSlRjV0dLN3FMZG1yL29iN2gzZW4wY1JORHVJcFJoVFdwY1NKWGllY3J1SEp0Y3JMRElnMzRmOGVRY0RhREdXcTVJbklaRkc3MU5ERTYvSUZham96blExK0ZJM0liY0c4eEtUemYxUllseG9jd2FBcmduWlFnMmhSc081VjJScjlRakp5cEcxeWYyWkI4M3Q1M2FsTTRCVVg1R1JZRFN6bitFeGNOSkphVjVIcTU2UDY0djRhU2VvQTBtQmRyU3BvV1Z5K0xwRElKWkFZcFFZZEQvc2ZrNC9sRU8xL0kwWTFRQkg1bU8zRWlkakVXa2hBZG05TGlLU3VvLzRLL21TcFBZTDB0S1l3cTlQRUUwTDVCSG9yQ2NiSTIvMm9VdG42NUNTOWZHeGpnUTRpZnJGZnE5VXhwbWZUTHZTZms4c3dmOXdCN2ZSTDRhN1E2SGh6MlF5enRqSDhNWFNnNHFXZTVobitBYUxYYWxHU1lWbkdRdnFGTlk3ODl2SnVUNnNuRXFrYjlTbTlVc3J3R3B4TzBnQUYyVG9MQWNScHlVeFE5QlMzS0M1aXdMaVM5OExHN0EzSFkxNnF4VU4yaW1lT2JEL3FFRXdLKzBsK0xQY

*Copy and run the output from previous cell*

In [19]:
!docker login -u AWS -p eyJwYXlsb2FkIjoiYnFQdjVIUkRpbzl0bXB4aFM4Ny9nbWp4OTV2UTZwTFp1WEhrNEZyWGsvaHBFNjBqUGU2Vm40aVNPdVhnb3BNaHNZMHJWeFFnQ09vUVBMRHdIbXdTSDR2TVZCdTUrL0gwb3V0Z3dJRFMwUUx6MmNxSmdPK0pqUmh4SWhDZTIwa0dHWWZ3M3gvM0pxYXlTcUdnUUtkSGFhMDJkenBnTWNOVU9ia2NESHJKQ0NJeGNZUGgwWHhiNWpCT0piYTlrU0RwajJIVFJCalNnMjZaRnZJUWVwcnprY0JpNStiUnMzVWFmbXozdHlHczJzTzI1SXh1QjFudzBiOVNIdjBXZTZydWVDSlRjV0dLN3FMZG1yL29iN2gzZW4wY1JORHVJcFJoVFdwY1NKWGllY3J1SEp0Y3JMRElnMzRmOGVRY0RhREdXcTVJbklaRkc3MU5ERTYvSUZham96blExK0ZJM0liY0c4eEtUemYxUllseG9jd2FBcmduWlFnMmhSc081VjJScjlRakp5cEcxeWYyWkI4M3Q1M2FsTTRCVVg1R1JZRFN6bitFeGNOSkphVjVIcTU2UDY0djRhU2VvQTBtQmRyU3BvV1Z5K0xwRElKWkFZcFFZZEQvc2ZrNC9sRU8xL0kwWTFRQkg1bU8zRWlkakVXa2hBZG05TGlLU3VvLzRLL21TcFBZTDB0S1l3cTlQRUUwTDVCSG9yQ2NiSTIvMm9VdG42NUNTOWZHeGpnUTRpZnJGZnE5VXhwbWZUTHZTZms4c3dmOXdCN2ZSTDRhN1E2SGh6MlF5enRqSDhNWFNnNHFXZTVobitBYUxYYWxHU1lWbkdRdnFGTlk3ODl2SnVUNnNuRXFrYjlTbTlVc3J3R3B4TzBnQUYyVG9MQWNScHlVeFE5QlMzS0M1aXdMaVM5OExHN0EzSFkxNnF4VU4yaW1lT2JEL3FFRXdLKzBsK0xQY2E3TjRacGpoaXJEamN6MFBCQ0hOMnNtSXJpUktFQUNSZlFnUGsyNTc4VzNlSlRZYk9DNnZGM29UV0ptbUNqN3RHSUpvd05TRDJNNzdoMWhYaFRpdzBaZ2FCTzdDTE5USTRWcHl4a2IxOE5IYWZ6TE5UTXdZeEhaanRqZDBaUkdzdTl4V0hJL1ZkL2hTbzJ4Uk9teTRMelhoV3k4S256ZUJwNXIwa2pvaWlDOHFSYmZvYlVyVnUvSmEyaklRSlh4K2xkekpJcyt6dlY0NHAyaWMrSzVsc3RFbVo5c05MUFQ4ZStRajVKU1oxSm1TMWZNcDBKSzhIV3pGdld1dlJIY0JBbWIwbGxVNlhrOUthd3JJNUdCNy9mQUc2S2RrdXdNbXRHNDlsSGV6OXgrT3Y0VmhvWEpuVGl3RkxSZnlWTEhRS1I4NXFBS3hGdkdFOUh4QUkrcXpiMUNJaDdrZ0R0RVY3RXpHV3IwN2ZzVjRiSExqMEhuckhoSVUwcEdzczNqRmoyQWxUZkpCMDNTZmc9PSIsImRhdGFrZXkiOiJBUUVCQUhqNmxjNFhJSncvN2xuMEhjMDBETWVrNkdFeEhDYlk0UklwVE1DSTU4SW5Vd0FBQUg0d2ZBWUpLb1pJaHZjTkFRY0dvRzh3YlFJQkFEQm9CZ2txaGtpRzl3MEJCd0V3SGdZSllJWklBV1VEQkFFdU1CRUVESnNTSCsvVjhDUStlUkhaWVFJQkVJQTdETThKR2JEc1h5NHBnR0pyQ3A2cm45Y0xiaDcrQ3NUOThiRGVvMXZwb3JyWVRQZEZXV0l2UUtITTJLTW9yYVJSTHcwZ0NzQk12ZjBrTEJFPSIsInZlcnNpb24iOiIyIiwidHlwZSI6IkRBVEFfS0VZIiwiZXhwaXJhdGlvbiI6MTU3NjU4NDg0OH0= https://192023623294.dkr.ecr.us-west-2.amazonaws.com


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded


**Build docker image**

In [59]:
!cd {saved_path} & docker build . --tag=192023623294.dkr.ecr.us-west-2.amazonaws.com/sentiment-ecs


Step 1/12 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/12 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Using cache
 ---> 52d60658abca
Step 3/12 : EXPOSE 5000
 ---> Using cache
 ---> 041d44f68694
Step 4/12 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> a618012fac78
Step 5/12 : RUN conda update conda -y       && conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> f40b70099ec8
Step 6/12 : COPY . /bento
 ---> be181a1904d3
Step 7/12 : WORKDIR /bento
 ---> Running in ea1152d959b1
Removing intermediate container ea1152d959b1
 ---> 60ff0402076b
Step 8/12 : RUN conda env update -n base -f /bento/environment.yml
 ---> Running in 10a070278bf7
Solving environment: ...working... done

python-3.7.2         | 31.9 MB   | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...wo

sh: line 1: cd: {saved_path}: No such file or directory


#### Create ECR repository

In [33]:
!aws ecr create-repository --repository-name sentiment-ecs

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:192023623294:repository/sentiment-ecs",
        "registryId": "192023623294",
        "repositoryName": "sentiment-ecs",
        "repositoryUri": "192023623294.dkr.ecr.us-west-2.amazonaws.com/sentiment-ecs",
        "createdAt": 1576542447.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        }
    }
}


In [34]:
!docker push 192023623294.dkr.ecr.us-west-2.amazonaws.com/sentiment-ecs

The push refers to repository [192023623294.dkr.ecr.us-west-2.amazonaws.com/sentiment-ecs]

96eea2fa: Preparing 
778aa2c1: Preparing 
c052c405: Preparing 
ab50d2e3: Preparing 
90513c25: Preparing 
2405333d: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 


90513c25: Pushing  877.5MB/1.13GBB90513c25: Pushed   1.138GB/1.13GBlatest: digest: sha256:516ea702785a656a85507b3717c83e8a1d4c901605c4fa301364c7c18fc30346 size: 2225


## Deploy to AWS ECS


#### 1. Install ECS-CLI tool
https://docs.aws.amazon.com/AmazonECS/latest/developerguide/ECS_CLI_installation.html

For Mac:
Download
```
sudo curl -o /usr/local/bin/ecs-cli https://amazon-ecs-cli.s3.amazonaws.com/ecs-cli-darwin-amd64-latest
```
Make it executable
```
sudo chmod +x /usr/local/bin/ecs-cli
```


#### 2. Configure ECS-CLI
https://docs.aws.amazon.com/AmazonECS/latest/developerguide/ECS_CLI_Configuration.html
Create cluster profile configuration
```
ecs-cli configure --cluster tutorial --default-launch-type FARGATE --config-name tutorial --region us-west-2
```
Create CLI profile
```
ecs-cli configure profile --access-key AWS_ACCESS_KEY_ID --secret-key AWS_SECRET_ACCESS_KEY --profile-name tutorial-profile
```

In [12]:
!ecs-cli configure --cluster tutorial --default-launch-type FARGATE --config-name tutorial --region us-west-2

INFO[0000] Saved ECS CLI cluster configuration tutorial. 


In [15]:
!ecs-cli configure profile --profile-name tutorial-profile --access-key AWS_ACCESS_KEY_ID --secret-key AWS_SECRET_ACCESS_KEY

INFO[0000] Saved ECS CLI profile configuration tutorial-profile. 


###  Create the Task Execution IAM Role

In [61]:
%%writefile task-execution-assume-role.json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "ecs-tasks.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

Overwriting task-execution-assume-role.json


In [62]:
!aws iam --region us-west-2 create-role --role-name ecsTaskExecutionRole --assume-role-policy-document file://task-execution-assume-role.json

{
    "Role": {
        "Path": "/",
        "RoleName": "ecsTaskExecutionRole",
        "RoleId": "AROASZNL76Z7C7Q7SZJ4D",
        "Arn": "arn:aws:iam::192023623294:role/ecsTaskExecutionRole",
        "CreateDate": "2019-12-17T01:04:08Z",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "",
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "ecs-tasks.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }
    }
}


In [64]:
!aws iam --region us-west-2 attach-role-policy --role-name ecsTaskExecutionRole --policy-arn arn:aws:iam::aws:policy/service-role/AmazonECSTaskExecutionRolePolicy

## Start up a AWS ECS Cluster

In [16]:
!ecs-cli up --cluster-config tutorial --ecs-profile tutorial-profile

INFO[0001] Created cluster                               cluster=tutorial region=us-west-2
INFO[0002] Waiting for your cluster resources to be created... 
INFO[0002] Cloudformation stack status                   stackStatus=CREATE_IN_PROGRESS
INFO[0063] Cloudformation stack status                   stackStatus=CREATE_IN_PROGRESS
VPC created: vpc-0465d14ba04402f80
Subnet created: subnet-0d23851806f3db403
Subnet created: subnet-0dece5451f1a3b8b2
Cluster creation succeeded.


**Use the VPC id from the output from previous cell**

In [44]:
!aws ec2 describe-security-groups --filters Name=vpc-id,Values=vpc-0465d14ba04402f80 --region us-west-2

{
    "SecurityGroups": [
        {
            "Description": "default VPC security group",
            "GroupName": "default",
            "IpPermissions": [
                {
                    "IpProtocol": "-1",
                    "IpRanges": [],
                    "Ipv6Ranges": [],
                    "PrefixListIds": [],
                    "UserIdGroupPairs": [
                        {
                            "GroupId": "sg-0258b891f053e077b",
                            "UserId": "192023623294"
                        }
                    ]
                }
            ],
            "OwnerId": "192023623294",
            "GroupId": "sg-0258b891f053e077b",
            "IpPermissionsEgress": [
                {
                    "IpProtocol": "-1",
                    "IpRanges": [
                        {
                            "CidrIp": "0.0.0.0/0"
                        }
                    ],
                    "Ipv6Ranges": [],
                    "Pre

**Use the security group ID from previous cell**

In [48]:
!aws ec2 authorize-security-group-ingress --group-id sg-0258b891f053e077b --protocol tcp --port 5000 --cidr 0.0.0.0/0 --region us-west-2


An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5000, to port: 5000, ALLOW" already exists


**Use the docker image information from docker push cell**

In [40]:
%%writefile docker-compose.yml
version: '3'
services:
  web:
    image: 192023623294.dkr.ecr.us-west-2.amazonaws.com/sentiment-ecs
    ports:
      - "5000:5000"
    logging:
      driver: awslogs
      options: 
        awslogs-group: sentiment-aws-ecs
        awslogs-region: us-west-2
        awslogs-stream-prefix: web

Overwriting docker-compose.yml


**Use the subnets from previous cell that create ECS cluster.**

**Use security group value from cell that describe secruity group cell**

In [50]:
%%writefile ecs-params.yml

version: 1
task_definition:
  task_execution_role: ecsTaskExecutionRole
  ecs_network_mode: awsvpc
  task_size:
    mem_limit: 0.5GB
    cpu_limit: 256
run_params:
  network_configuration:
    awsvpc_configuration:
      subnets:
        - subnet-0d23851806f3db403
        - subnet-0dece5451f1a3b8b2
      security_groups:
        - sg-0258b891f053e077b
      assign_public_ip: ENABLED

Overwriting ecs-params.yml


In [65]:
!ecs-cli compose --project-name tutorial-bentoml-ecs service up --create-log-groups --cluster-config tutorial --ecs-profile tutorial-profile

INFO[0000] Using ECS task definition                     TaskDefinition="tutorial-bentoml-ecs:1"
WARN[0001] Failed to create log group sentiment-aws-ecs in us-west-2: The specified log group already exists 
INFO[0001] Updated ECS service successfully              desiredCount=1 force-deployment=false service=tutorial-bentoml-ecs
INFO[0017] (service tutorial-bentoml-ecs) has started 1 tasks: (task ecd119f0-b159-42e6-b86c-e6a62242ce7a).  timestamp="2019-12-17 01:05:23 +0000 UTC"
INFO[0094] Service status                                desiredCount=1 runningCount=1 serviceName=tutorial-bentoml-ecs
INFO[0094] (service tutorial-bentoml-ecs) has reached a steady state.  timestamp="2019-12-17 01:06:40 +0000 UTC"
INFO[0094] ECS Service has reached a stable state        desiredCount=1 runningCount=1 serviceName=tutorial-bentoml-ecs


In [68]:
!ecs-cli compose --project-name tutorial-bentoml-ecs service ps --cluster-config tutorial --ecs-profile tutorial-profile

Name                                      State    Ports                        TaskDefinition          Health
ecd119f0-b159-42e6-b86c-e6a62242ce7a/web  RUNNING  34.212.49.46:5000->5000/tcp  tutorial-bentoml-ecs:1  UNKNOWN


### Test ECS endpoint

In [71]:
!curl -i \
--request POST \
--header "Content-Type: application/json" \
--data '["sweet food", "bad food", "happy day"]' \
http://34.212.49.46:5000/predict









[4, 0, 4]

# Clean up ECS deployment

In [72]:
!ecs-cli compose --project-name tutorial-bentoml-ecs service down --cluster-config tutorial --ecs-profile tutorial-profile

INFO[0000] Updated ECS service successfully              desiredCount=0 force-deployment=false service=tutorial-bentoml-ecs
INFO[0000] Service status                                desiredCount=0 runningCount=1 serviceName=tutorial-bentoml-ecs
INFO[0016] Service status                                desiredCount=0 runningCount=0 serviceName=tutorial-bentoml-ecs
INFO[0016] (service tutorial-bentoml-ecs) has stopped 1 running tasks: (task ecd119f0-b159-42e6-b86c-e6a62242ce7a).  timestamp="2019-12-17 01:15:37 +0000 UTC"
INFO[0016] ECS Service has reached a stable state        desiredCount=0 runningCount=0 serviceName=tutorial-bentoml-ecs
INFO[0016] Deleted ECS service                           service=tutorial-bentoml-ecs
INFO[0016] ECS Service has reached a stable state        desiredCount=0 runningCount=0 serviceName=tutorial-bentoml-ecs


In [73]:
!ecs-cli down --force --cluster-config tutorial --ecs-profile tutorial-profile

INFO[0001] Waiting for your cluster resources to be deleted... 
INFO[0001] Cloudformation stack status                   stackStatus=DELETE_IN_PROGRESS
INFO[0062] Deleted cluster                               cluster=tutorial
